In [ ]:
import torch
import pickle
import argparse
import numpy as np
import torch.nn as nn
from torchmetrics import Accuracy
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import os
from fastai.vision.all import untar_data, URLs
from dime.data_utils import MaskLayerGaussian, MaskLayer2d
from dime import GreedyCMIEstimatorPL
from dime.masking_pretrainer import MaskingPretrainer
from dime.resnet_imagenet import resnet18, Predictor, ValueNetwork, ResNet18Backbone, resnet50
from dime.utils import normalize, selection_with_lamda
from dime.vit import PredictorViT, ValueNetworViT
import timm
import matplotlib.pyplot as plt

# Get Dataset

In [ ]:
acc_metric = Accuracy(task='multiclass', num_classes=100)

image_size = 224
mask_width = 14
mask_type = 'zero'
if mask_type == 'gaussian':
    mask_layer = MaskLayerGaussian(append=False, mask_width=mask_width, patch_size=image_size/mask_width)
else:
    mask_layer = MaskLayer2d(append=False, mask_width=mask_width, patch_size=image_size/mask_width)

device = torch.device('cuda:5')

norm_constants = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

transforms_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(*norm_constants),
    ])

dataset_path = "/projects/<labname>/<username>/ImageNet100"
test_dataset = ImageFolder(dataset_path+'/val.X', transforms_test)

# Prepare dataloaders.
mbsize = 32
test_dataloader = DataLoader(test_dataset, batch_size=mbsize, pin_memory=True, drop_last=True, num_workers=4)


# Load pretrained model

In [ ]:
arch = "vit_small_patch16_224"

if arch in ['resnet18']:
    backbone, expansion = ResNet18Backbone(eval(arch + '(pretrained=True)'))
    predictor =  Predictor(backbone, expansion, num_classes=100)
    block_layer_stride = 1
    if mask_width == 14:
        block_layer_stride = 0.5
    value_network = ValueNetwork(backbone, expansion, block_layer_stride=block_layer_stride)
    predictor.load_state_dict(torch.load(f"results/predictor_trained_max_features_50_{arch}_lr_1e-5_use_entropy_with_token_individual_uniform_feature_cost_zero_mask_width_14.pth"))
    value_network.load_state_dict(torch.load(f"results/value_network_trained_max_features_50_{arch}_lr_1e-5_use_entropy_with_token_individual_uniform_feature_cost_zero_mask_width_14.pth"))
else:
    backbone = timm.create_model(arch, pretrained=True)
    predictor =  PredictorViT(backbone, num_classes=100)
    value_network = ValueNetworViT(backbone, mask_width=mask_width)
    predictor.load_state_dict(torch.load(f"results/predictor_trained_max_features_50_vit_small_patch16_224_lr_1e-5_use_entropy_sigmoid_with_token_individual_uniform_feature_cost_zero_mask_width_14.pth"))
    value_network.load_state_dict(torch.load(f"results/value_network_trained_max_features_50_vit_small_patch16_224_lr_1e-5_use_entropy_sigmoid_with_token_individual_uniform_feature_cost_zero_mask_width_14.pth"))
    
greedy_cmi_estimator = GreedyCMIEstimator(value_network, predictor, mask_layer).to(device)

trainer = Trainer(
                    accelerator='gpu',
                    devices=[device.index],
                    precision=16
                )

# Evaluate Penalized Policy

In [ ]:
avg_num_features_lamda = []
accuracy_scores_lamda = []
all_masks_lamda =[]

lamda_values = list(np.geomspace(0.005, 0.03, num=12))
for lamda in [0.04]:
    metric_dict = greedy_cmi_estimator.inference(trainer, test_dataloader, feature_costs=None, lam=lamda)
        
    y = metric_dict['y']
    pred = metric_dict['pred']
    accuracy_score = acc_metric(pred, y)
    final_masks = np.array(metric_dict['mask'])
    accuracy_scores_lamda.append(accuracy_score)
    avg_num_features_lamda.append(np.mean(np.sum(final_masks, axis=1)))
    results_dict['acc'][np.mean(np.sum(final_masks, axis=1))] = accuracy_score

    print(f"Lambda={lamda}, Acc={accuracy_score}, Avg. num features={np.mean(np.sum(final_masks, axis=1))}")
    all_masks_lamda.append(final_masks)

# Evaluate Budget Constrained Policy

In [ ]:
avg_num_features_budget = []
accuracy_scores_budget = []
all_masks_budget=[]

max_budget_values = [18] + list(range(5, 40, 5))
for budget in max_budget_values:
    metric_dict_budget = greedy_cmi_estimator.inference(trainer, test_dataloader, feature_costs=None, budget=budget)
        
    y = metric_dict_budget['y']
    pred = metric_dict_budget['pred']
    accuracy_score = acc_metric(pred, y)
    final_masks = np.array(metric_dict_budget['mask'])
    accuracy_scores_budget.append(accuracy_score)
    avg_num_features_budget.append(np.mean(np.sum(final_masks, axis=1)))
    results_dict['acc'][np.mean(np.sum(final_masks, axis=1))] = accuracy_score
    print(f"Budget={budget}, Acc={accuracy_score}, Avg. num features={np.mean(np.sum(final_masks, axis=1))}")

    all_masks_budget.append(final_masks)

# Evaluate Confidence Constrained Policy

In [ ]:
avg_num_features_confidence = []
accuracy_scores_confidence = []
all_masks_confidence=[]
confidence_values = list(np.arange(0.1, 1, 0.1))

for confidence in confidence_values:
    metric_dict = greedy_cmi_estimator.inference(trainer, test_dataloader, feature_costs=None, confidence=confidence)
    y = metric_dict['y']
    pred = metric_dict'pred']
    accuracy_score = acc_metric(pred, y)

    final_masks = np.array(metric_dict['mask'])
    accuracy_scores_confidence.append(accuracy_score)
    avg_num_features_confidence.append(np.mean(np.sum(final_masks, axis=1)))
    print(f"Confidence={confidence}, Acc={accuracy_score}, Avg. num features={np.mean(np.sum(final_masks, axis=1))}")
    all_masks_confidence.append(final_masks)